# Baseline - AutoGluon without Deep Learning Models

you need to create indpenendt venv becuase it does not work with global one

source .autogluon_venv/bin/actiavte

source /workspace/soil-ml-modeling-pipeline/.autogluon_venv/bin/activate

pip install autogluon.timeseries

pip install polars

In [ ]:
import polars as pl
import pandas as pd
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

in the future you can add known_covariates https://auto.gluon.ai/stable/api/autogluon.timeseries.TimeSeriesPredictor.predict.html#timeseriespredictor-predict



In [1]:
import polars as pl
df = pl.read_parquet('/workspace/soil-ml-modeling-pipeline/ml-modeling-pipeline/data/02_intermediate/preprocessed_data.parquet')
df

lat,lon,time,T2M,T2M_MAX,T2M_MIN,TS,PS,WS2M,WS10M,WS50M,QV2M,QV10M,RH2M,TQV,T2MDEW,T2MWET,DISPH,RHOA,TO3,TROPPB,TROPQ,TROPT,U10M,U2M,U50M,V10M,V2M,V50M,WD10M,WD2M,WD50M,Nino12,Nino3,Nino34,Nino4,eddi,PRECTOTCORR_mm,eddi_category,id,id_right
f64,f64,date,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f32,f32,f32,f32,f32,f64,str,u32,u32
-11.5,35.0,1981-11-30,25.326563,27.779688,23.25625,299.523438,92086.625,1.890625,2.3984375,3.09375,0.015503,0.015015,71.125,45.234375,292.695312,295.585938,1.359375,1.067383,267.460938,10051.5625,0.0,190.671875,-1.117188,-0.820312,-1.59375,-1.5625,-1.21875,-1.960938,68.1875,66.3125,59.8125,0.05473,-0.033769,-0.033502,-0.087432,-0.761296,8.020501,"""ED2""",1598,270
-11.5,35.0,1981-12-01,24.709375,27.44375,22.576563,299.59375,92232.75,2.484375,3.0546875,3.71875,0.01532,0.014832,72.75,46.632812,292.515625,295.1875,1.3671875,1.072266,270.132812,10191.25,0.0,191.890625,-1.132812,-0.875,-1.570312,1.2109375,1.0234375,1.3828125,124.8125,124.6875,123.6875,0.125351,-0.068272,-0.045651,-0.085928,-0.761296,20.987721,"""ED2""",1598,270
-11.5,35.0,1981-12-02,23.35,25.584375,22.0375,297.820312,92331.1875,2.9140625,3.6328125,4.4765625,0.015442,0.014954,79.3125,47.429688,292.648438,294.570312,1.3828125,1.078125,269.914062,10124.4375,0.0,190.570312,-3.265625,-2.59375,-4.054688,1.4921875,1.2265625,1.78125,118.3125,119.75,116.5625,0.20493,-0.132351,-0.089331,-0.061235,-0.761296,6.153899,"""ED2""",1598,270
-11.5,35.0,1981-12-03,21.6625,24.264063,20.053125,296.664062,92429.5,3.96875,4.875,5.8359375,0.013306,0.012878,75.9375,43.054688,290.28125,292.546875,1.390625,1.086914,267.726562,10258.9375,0.0,190.515625,-3.804688,-3.085938,-4.601562,2.734375,2.2265625,3.21875,126.3125,126.375,126.0625,-0.08833,-0.228514,-0.27863,-0.166646,-0.841127,0.030492,"""ED3""",1598,270
-11.5,35.0,1981-12-04,23.514063,27.63125,19.576563,298.421875,92405.875,3.6953125,4.53125,5.4453125,0.01355,0.013123,69.9375,37.070312,290.625,293.640625,1.40625,1.081055,267.429688,10843.25,0.0,193.03125,-3.515625,-2.789062,-4.445312,1.90625,1.6015625,2.109375,122.125,123.5625,119.6875,-0.346005,-0.255113,-0.333117,-0.287201,-1.019687,0.775886,"""ED3""",1598,270
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
22.0,36.25,2024-04-25,32.654688,40.279688,25.146875,309.5,96249.3125,2.7109375,3.3828125,4.015625,0.010803,0.010681,35.5,30.789062,287.695312,296.75,0.0234375,1.092773,287.734375,8693.9375,0.0,195.835938,-3.085938,-2.476562,-3.59375,1.03125,0.8203125,1.4765625,112.3125,112.75,113.9375,0.953016,1.714916,1.595976,1.564177,0.978788,0.061682,"""EW3""",1192,350
22.0,36.25,2024-04-26,32.685938,40.201563,26.623438,309.945312,96206.75,2.1015625,2.6796875,3.1640625,0.01123,0.011108,37.75,25.96875,288.21875,297.023438,0.0234375,1.091797,295.875,10718.0625,0.0,200.460938,-1.734375,-1.328125,-2.101562,-1.398438,-1.171875,-1.507812,69.5,69.625,68.4375,0.988236,1.717664,1.600204,1.535523,0.978788,0.003058,"""EW3""",1192,350
22.0,36.25,2024-04-27,30.389063,38.529688,24.857813,307.820312,96348.8125,3.0625,3.7421875,4.2109375,0.013123,0.012939,50.1875,33.5,290.609375,297.078125,0.0234375,1.102539,298.421875,10708.5625,0.000008,200.226562,-2.335938,-1.921875,-2.648438,-2.25,-1.835938,-2.367188,91.0,90.8125,78.75,1.050644,1.706506,1.607002,1.507596,0.943385,5.993525,"""EW3""",1192,350


In [9]:
import polars as pl

# Assuming 'df' is your Polars DataFrame and 'eddi_category' is your target column
# Cast the column to categorical to optimize performance and enable category codes
df = df.with_columns(df['eddi_category'].cast(pl.Categorical))

# Extract category codes directly using Polars
df = df.with_columns(df['eddi_category'].cast(pl.UInt32).alias('eddi_category_encoded'))

In [10]:
df

lat,lon,time,T2M,T2M_MAX,T2M_MIN,TS,PS,WS2M,WS10M,WS50M,QV2M,QV10M,RH2M,TQV,T2MDEW,T2MWET,DISPH,RHOA,TO3,TROPPB,TROPQ,TROPT,U10M,U2M,U50M,V10M,V2M,V50M,WD10M,WD2M,WD50M,Nino12,Nino3,Nino34,Nino4,eddi,PRECTOTCORR_mm,eddi_category,id,id_right,eddi_category_encoded
f64,f64,date,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f32,f32,f32,f32,f32,f64,cat,u32,u32,u32
-11.5,35.0,1981-11-30,25.326563,27.779688,23.25625,299.523438,92086.625,1.890625,2.3984375,3.09375,0.015503,0.015015,71.125,45.234375,292.695312,295.585938,1.359375,1.067383,267.460938,10051.5625,0.0,190.671875,-1.117188,-0.820312,-1.59375,-1.5625,-1.21875,-1.960938,68.1875,66.3125,59.8125,0.05473,-0.033769,-0.033502,-0.087432,-0.761296,8.020501,"""ED2""",1598,270,0
-11.5,35.0,1981-12-01,24.709375,27.44375,22.576563,299.59375,92232.75,2.484375,3.0546875,3.71875,0.01532,0.014832,72.75,46.632812,292.515625,295.1875,1.3671875,1.072266,270.132812,10191.25,0.0,191.890625,-1.132812,-0.875,-1.570312,1.2109375,1.0234375,1.3828125,124.8125,124.6875,123.6875,0.125351,-0.068272,-0.045651,-0.085928,-0.761296,20.987721,"""ED2""",1598,270,0
-11.5,35.0,1981-12-02,23.35,25.584375,22.0375,297.820312,92331.1875,2.9140625,3.6328125,4.4765625,0.015442,0.014954,79.3125,47.429688,292.648438,294.570312,1.3828125,1.078125,269.914062,10124.4375,0.0,190.570312,-3.265625,-2.59375,-4.054688,1.4921875,1.2265625,1.78125,118.3125,119.75,116.5625,0.20493,-0.132351,-0.089331,-0.061235,-0.761296,6.153899,"""ED2""",1598,270,0
-11.5,35.0,1981-12-03,21.6625,24.264063,20.053125,296.664062,92429.5,3.96875,4.875,5.8359375,0.013306,0.012878,75.9375,43.054688,290.28125,292.546875,1.390625,1.086914,267.726562,10258.9375,0.0,190.515625,-3.804688,-3.085938,-4.601562,2.734375,2.2265625,3.21875,126.3125,126.375,126.0625,-0.08833,-0.228514,-0.27863,-0.166646,-0.841127,0.030492,"""ED3""",1598,270,1
-11.5,35.0,1981-12-04,23.514063,27.63125,19.576563,298.421875,92405.875,3.6953125,4.53125,5.4453125,0.01355,0.013123,69.9375,37.070312,290.625,293.640625,1.40625,1.081055,267.429688,10843.25,0.0,193.03125,-3.515625,-2.789062,-4.445312,1.90625,1.6015625,2.109375,122.125,123.5625,119.6875,-0.346005,-0.255113,-0.333117,-0.287201,-1.019687,0.775886,"""ED3""",1598,270,1
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
22.0,36.25,2024-04-25,32.654688,40.279688,25.146875,309.5,96249.3125,2.7109375,3.3828125,4.015625,0.010803,0.010681,35.5,30.789062,287.695312,296.75,0.0234375,1.092773,287.734375,8693.9375,0.0,195.835938,-3.085938,-2.476562,-3.59375,1.03125,0.8203125,1.4765625,112.3125,112.75,113.9375,0.953016,1.714916,1.595976,1.564177,0.978788,0.061682,"""EW3""",1192,350,9
22.0,36.25,2024-04-26,32.685938,40.201563,26.623438,309.945312,96206.75,2.1015625,2.6796875,3.1640625,0.01123,0.011108,37.75,25.96875,288.21875,297.023438,0.0234375,1.091797,295.875,10718.0625,0.0,200.460938,-1.734375,-1.328125,-2.101562,-1.398438,-1.171875,-1.507812,69.5,69.625,68.4375,0.988236,1.717664,1.600204,1.535523,0.978788,0.003058,"""EW3""",1192,350,9
22.0,36.25,2024-04-27,30.389063,38.529688,24.857813,307.820312,96348.8125,3.0625,3.7421875,4.2109375,0.013123,0.012939,50.1875,33.5,290.609375,297.078125,0.0234375,1.102539,298.421875,10708.5625,0.000008,200.226562,-2.335938,-1.921875,-2.648438,-2.25,-1.835938,-2.367188,91.0,90.8125,78.75,1.050644,1.706506,1.607002,1.507596,0.943385,5.993525,"""EW3""",1192,350,9


In [ ]:
train_data = TimeSeriesDataFrame.from_data_frame(
    df,
    id_column="id",
    timestamp_column="time"
)
train_data.head()

In [ ]:
predictor = TimeSeriesPredictor(
    prediction_length=90,
    path="autogluon-m4-hourly",
    target="eddi_category",
    eval_metric="MASE",
)

In [ ]:
predictor.fit(
    train_data,
    presets="medium_quality",
    time_limit=600,
)